In [25]:
# Import general python packages
import pandas as pd
import numpy as np
import seaborn as sns
import copy

# Import functions
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over, sum_over_time

# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Scenario,
                           Constraint, ScalarOutcome)
from ema_workbench.util import ema_logging
from ema_workbench import save_results, load_results
from ema_workbench.em_framework.optimization import (EpsilonProgress)

In [26]:
experiments, outcomes = load_results('Experiments/try_DisagT_PD7_1_1.tar.gz')

[MainProcess/INFO] results loaded successfully from C:\Users\Gebruiker\OneDrive - Delft University of Technology\EPA\Q4 Model based decision making\epa141A_open\final_ass_time_changed\Experiments\try_DisagT_PD7_1_1.tar.gz


In [27]:
outcomes

{'Expected Number of Deaths': array([[0.00062459, 0.        , 0.        , 0.        , 0.        ]]),
 'A.1_Expected Annual Damage': array([[0., 0., 0., 0., 0.]]),
 'A.2_Expected Annual Damage': array([[3558948.76099897,       0.        ,       0.        ,
               0.        ,       0.        ]]),
 'A.3_Expected Annual Damage': array([[0., 0., 0., 0., 0.]]),
 'A.4_Expected Annual Damage': array([[0., 0., 0., 0., 0.]]),
 'A.5_Expected Annual Damage': array([[0., 0., 0., 0., 0.]]),
 'Total_period_Costs': array([[7.49191029e+08, 2.91567221e+08, 3.97296929e+08, 2.26396419e+08,
         8.45652241e+08]])}

In [28]:
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 4,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,A.5_DikeIncrease 4,EWS_DaysToThreat,scenario,policy,model
0,87,339.18835,1.0,0.954041,266.819102,1.0,0.230765,86.963423,1.0,0.221302,...,6,2,2,1,8,7,2,1,0,dikesnet


# MORDM

In [2]:
# Loading in all the 17 objectives via predefined problem formulation 3
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)
    dike_model, planning_steps = get_model_for_problem_formulation(7)

In [3]:
print(planning_steps)

[0 1 2 3 4]


In [4]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(repr(outcome))
    

ArrayOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over_time at 0x00000199F5707BA0>)
ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('Total_period_Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over_time at 0x00000199F5707BA0>)


In [10]:
# Writing a function to create actor specific problem formulations
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers
    
    cost_variables = []
    cost_variables.extend(
    [
        f"{dike}_{e}"
        for e in ["Expected Annual Damage", "Dike Investment Costs"]
        for dike in function.dikelist
    ])
    cost_variables.extend([f"RfR Total Costs"])
    cost_variables.extend([f"Expected Evacuation Costs"])


    if problem_formulation_actor == 4: #RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'
                                                                                ] + ['Expected Evacuation Costs'],
                            function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)] 
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1-4',
                            variable_name=['A.1_Expected Annual Damage' ,'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage'], function=sum_over, kind=direction),

            ScalarOutcome('Investment Costs A1-4',
                            variable_name=['A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs'], function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths in A1-4',
                            variable_name=['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths'], function=sum_over, kind=direction)]
    
    elif problem_formulation_actor == 6: # OVERIJSSEL
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A4', variable_name='A.4_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A5', variable_name='A.5_Expected Annual Damage', function = sum_over, kind=direction),            
            ScalarOutcome('Expected Number of Deaths in A4', variable_name='A.4_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A5', variable_name='A.5_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction),
            ScalarOutcome('Aggregated Expected Number of Deaths A1-A3', variable_name =
            ['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 
             'A.3_Expected Number of Deaths'], function = sum_over, kind=direction)]
    elif problem_formulation_actor == 7: 

    
    else:
        raise TypeError('unknown identifier')
    return model

In [11]:
# Replicate the uncertainties
uncertainties = dike_model.uncertainties
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [12]:
levers = dike_model.levers 
levers = copy.deepcopy(dike_model.levers)

In [13]:
model = problem_formulation_actor(7)
for outcome in model.outcomes:
    print(repr((outcome)))

ArrayOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over_time at 0x00000199F5707BA0>)
ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('Total_period_Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over_time at 0x00000199F5707BA0>)


In [14]:
print(len(model.outcomes))

7


In [15]:
reference_values = {
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.5,
    "ID flood wave shape": 4,
    "planning steps": 2,
}
reference_values.update({f"discount rate {n}": 3.5 for n in planning_steps})
refcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        refcase_scen.update({key.name: reference_values[key.name]})
    else:
        refcase_scen.update({key.name: reference_values[name_split[1]]})
            
ref_scenario = Scenario('reference', **refcase_scen)

In [16]:
convergence_metrics = {EpsilonProgress()}
#constraint = [Constraint("Total Costs", outcome_names="Total Costs", function=lambda x: max(0, x - 7700000000))]

with MultiprocessingEvaluator(model,n_processes=-1) as evaluator:
   results2 = evaluator.optimize(nfe=1, searchover='levers',
                                 convergence=convergence_metrics,
                                 epsilons=[1]*len(model.outcomes), reference=ref_scenario)

save_results(results2, 'Experiments/Week23_MORDM_Reference_1000_PD6.tar.gz')


[MainProcess/INFO] pool started with 3 workers
[MainProcess/INFO] terminating pool


EMAError: No outcomes specified to optimize over, all outcomes are of kind=INFO

In [106]:
y,t = results2

In [107]:
y

### Worst Case

In [18]:
# Worst case specification
worstcase_values ={
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.9,
    "ID flood wave shape": 4,
    "planning steps": 2,
}
worstcase_values.update({f"discount rate {n}": 3.5 for n in planning_steps})

worstcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        worstcase_scen.update({key.name: worstcase_values[key.name]})
    else:
        worstcase_scen.update({key.name: worstcase_values[name_split[1]]})
            
worst_scenario = Scenario('reference', **worstcase_scen)

In [21]:
#convergence_metrics = {EpsilonProgress()}

#with MultiprocessingEvaluator(model,n_processes=-1) as evaluator:
  #  results3 = evaluator.optimize(nfe=4000, searchover='levers',
     #                            convergence=convergence_metrics,
      #                           epsilons=[0.01]*len(model.outcomes), reference=worst_scenario)

#save_results(results2, 'Experiments/Week23_MORDM_Reference_1000_PD6.tar.gz')


In [23]:
worst_case.to_csv('MODRM_WC.csv')

In [22]:
# Selecting the policies that are proposed as solutions for further robustness tests
from ema_workbench import Policy

worstcase_policies_to_evaluate = []

for i, policy in import_worst_case.iterrows():
    worstcase_policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [23]:
n_scenarios = 2000
#with MultiprocessingEvaluator(model) as evaluator:
 #   worst_results = evaluator.perform_experiments(n_scenarios,
  #                                          worstcase_policies_to_evaluate)

In [24]:
save_results(worst_results, 'Week23_worst_case_2000_4.tar.gz')

### Reference case

In [25]:
reference_case_re = pd.read_csv('MODRM_DF.csv')

In [26]:
# Selecting the policies that are proposed as solutions for further robustness tests
from ema_workbench import Policy

refercase_policies_to_evaluate = []

for i, policy in reference_case_re.iterrows():
    refercase_policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [27]:
n_scenarios = 2000
with MultiprocessingEvaluator(model) as evaluator:
    reference_policies_results = evaluator.perform_experiments(n_scenarios,
                                            refercase_policies_to_evaluate)

In [28]:
save_results(reference_policies_results, 'Week23_reference_case_2000_6.tar.gz')

# MODRM GRAPHS


In [90]:
import pandas as pd

# Correct way to read a CSV file into a DataFrame
rf_policy_set= pd.read_csv('MODRM_DF.csv')


In [94]:
rf_policy_set

In [105]:
from ema_workbench.analysis import parcoords

ref_data = rf_policy_set.loc[:, [o.name for o in model.outcomes]]
ref_limits = parcoords.get_limits(ref_data)
ref_limits.loc[0, ['Expected Annual Damage A4', 'Expected Annual Damage A5','Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']] = 0

paraxes = parcoords.ParallelAxes(ref_limits)
paraxes.plot(ref_data)
plt.title('Trade-offs in Reference case scenario')

# Set figure size
# Run this twice, to get a nice broad view
fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size
plt.show()


In [129]:
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench.analysis import parcoords

# Assuming WC_policy_set and model are already defined
ref_data = rf_policy_set.loc[:, [o.name for o in model.outcomes]]
ref_limits = parcoords.get_limits(ref_data)
ref_limits.loc[0, ['Expected Annual Damage A4', 'Expected Annual Damage A5',
                   'Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']] = 0

# Create a ParallelAxes object
paraxes = parcoords.ParallelAxes(ref_limits)

# Generate a color palette
colors = sns.color_palette('husl', len(rf_policy_set))

# Plot each policy individually with a corresponding color and label
for i, (index, row) in enumerate(rf_policy_set.iterrows()):
    outcomes = row.loc[['Expected Annual Damage A4', 'Expected Annual Damage A5',
                        'Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']]
    paraxes.plot(outcomes.to_frame().T, color=colors[i], label=f'Policy {index}')

# Add a legend to the plot
paraxes.legend()


In [109]:
import pandas as pd

# Correct way to read a CSV file into a DataFrame
WC_policy_set= pd.read_csv('MODRM_WC.csv')


In [110]:
WC_policy_set

In [125]:
from ema_workbench.analysis import parcoords

ref_data = WC_policy_set.loc[:, [o.name for o in model.outcomes]]
ref_limits = parcoords.get_limits(ref_data)
ref_limits.loc[0, ['Expected Annual Damage A4', 'Expected Annual Damage A5','Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']] = 0

paraxes = parcoords.ParallelAxes(ref_limits)
paraxes.plot(ref_data)
plt.title('Trade-offs in Worst case scenario')
paraxes.legend()
# Set figure size
# Run this twice, to get a nice broad view
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size
plt.show()



In [130]:
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench.analysis import parcoords

# Assuming WC_policy_set and model are already defined
ref_data = WC_policy_set.loc[:, [o.name for o in model.outcomes]]
ref_limits = parcoords.get_limits(ref_data)
ref_limits.loc[0, ['Expected Annual Damage A4', 'Expected Annual Damage A5',
                   'Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']] = 0

# Create a ParallelAxes object
paraxes = parcoords.ParallelAxes(ref_limits)

# Generate a color palette
colors = sns.color_palette('husl', len(WC_policy_set))

# Plot each policy individually with a corresponding color and label
for i, (index, row) in enumerate(WC_policy_set.iterrows()):
    outcomes = row.loc[['Expected Annual Damage A4', 'Expected Annual Damage A5',
                        'Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']]
    paraxes.plot(outcomes.to_frame().T, color=colors[i], label=f'Policy {index}')

# Add a legend to the plot
paraxes.legend()

# Set the figure size
plt.figure(figsize=(12, 12))



# PRIM

In [136]:
rf_em = load_results('Week23_reference_case_2000_6.tar.gz')

In [140]:
experiments_rfem, outcomes_rfem = rf_em

In [172]:
experiments_rfem[:, 

In [171]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming outcomes_rfem is a tuple containing a DataFrame and a dictionary
experiments_df, outcomes_dict = rf_em

# Extract the policy data
policies = experiments_df['policy']

# Create a DataFrame from the outcomes dictionary and add the policy column
outcomes_df = pd.DataFrame(outcomes_dict)
outcomes_df['policy'] = policies.values

# Calculate the mean of each outcome for each policy
mean_outcomes = outcomes_df.groupby('policy').mean()

# Plot the mean outcomes against the policies
mean_outcomes.plot(kind='bar', figsize=(12, 8))
plt.title('Mean Outcomes by Policy')
plt.xlabel('Policy')
plt.ylabel('Mean Value')
plt.legend(title='Outcomes')
plt.show()


In [176]:
policiess = experiments_rfem.iloc[:,19:]
policiess

In [179]:
scenaaa =experiments_rfem.iloc[:,:19]
scenaaa

In [166]:
outcomes_rfem['Expected Annual Damage A4']

In [141]:
outcomes_rfem

In [152]:
from ema_workbench.analysis import prim
import matplotlib.pyplot as plt

data = outcomes_rfem['Total Costs']

# With continious outcomes, probably the only way to prim is throufh percentile??

y = data < np.percentile(data, 10)

# Initialize PRIM algorithm with the appropriate mode
prim_alg = prim.Prim(experiments_rfem, y, threshold=0.7)
box1 = prim_alg.find_box()

box1.show_tradeoff(annotated=True)
plt.show()

In [163]:
point = 52
box1.inspect(point)

In [182]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming outcomes_rfem is a tuple containing a DataFrame and a dictionary
experiments_df, outcomes_dict = rf_em

# Extract the policy data
policies = experiments_df['policy']

# Create a DataFrame from the outcomes dictionary and add the policy column
outcomes_df = pd.DataFrame(outcomes_dict)
outcomes_df['policy'] = policies.values

# Calculate the mean and standard deviation of each outcome for each policy
mean_outcomes = outcomes_df.groupby('policy').mean()
std_outcomes = outcomes_df.groupby('policy').std()

# Plot the mean outcomes against the policies
mean_outcomes.plot(kind='bar', figsize=(14, 8))
plt.title('Mean Outcomes by Policy')
plt.xlabel('Policy')
plt.ylabel('Mean Value')
plt.legend(title='Outcomes')
plt.show()


In [183]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming outcomes_rfem is a tuple containing a DataFrame and a dictionary
experiments_df, outcomes_dict = rf_em

# Extract the policy data
policies = experiments_df['policy']

# Create a DataFrame from the outcomes dictionary and add the policy column
outcomes_df = pd.DataFrame(outcomes_dict)
outcomes_df['policy'] = policies.values

# Calculate the mean and standard deviation of each outcome for each policy
mean_outcomes = outcomes_df.groupby('policy').mean()
std_outcomes = outcomes_df.groupby('policy').std()

# Plot the mean outcomes for each policy separately
for outcome in mean_outcomes.columns:
    plt.figure(figsize=(10, 6))
    mean_outcomes[outcome].plot(kind='bar')
    plt.title(f'Mean {outcome} by Policy')
    plt.xlabel('Policy')
    plt.ylabel(f'Mean {outcome}')
    plt.show()

# Plot the standard deviation outcomes for each policy separately
for outcome in std_outcomes.columns:
    plt.figure(figsize=(10, 6))
    std_outcomes[outcome].plot(kind='bar')
    plt.title(f'Standard Deviation of {outcome} by Policy')
    plt.xlabel('Policy')
    plt.ylabel(f'Standard Deviation of {outcome}')
    plt.show()

# Plot boxplots for each outcome grouped by policy
for outcome in outcomes_df.columns[:-1]:  # Exclude the 'policy' column
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='policy', y=outcome, data=outcomes_df)
    plt.title(f'Boxplot of {outcome} by Policy')
    plt.xlabel('Policy')
    plt.ylabel(outcome)
    plt.show()

# Plot histograms for each outcome grouped by policy
for outcome in outcomes_df.columns[:-1]:  # Exclude the 'policy' column
    plt.figure(figsize=(10, 6))
    for policy in outcomes_df['policy'].unique():
        subset = outcomes_df[outcome][outcomes_df['policy'] == policy]
        sns.histplot(subset, kde=True, label=f'Policy
